In [1]:
#import necessary libraries
import pandas as pd
import nltk
import sys # needed this for certain print options during debugging
import numpy as np #lmao why did we not have this before
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import json # since we have a json file this time as our dataset.
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/soumyankar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
steam_2022 = pd.read_json('data/steam_games.json')
steam_2022 = steam_2022.T.reset_index()
steam_2022.drop("index", axis=1, inplace=True)
steam_cols=steam_2022.columns.to_list()
print(steam_cols)
# print(steam_2022.head(2))

['appid', 'name', 'short_description', 'developer', 'publisher', 'genre', 'tags', 'type', 'categories', 'owners', 'positive', 'negative', 'price', 'initialprice', 'discount', 'ccu', 'languages', 'platforms', 'release_date', 'required_age', 'website', 'header_image']


In [3]:
from IPython.display import display, HTML
display(steam_2022.sample(5))

,appid,name,short_description,developer,publisher,genre,tags,type,categories,owners,positive,negative,price,initialprice,discount,ccu,languages,platforms,release_date,required_age,website,header_image
53443,934180,Trials of Wilderness,"You are alone on an island, trying desperately to survive amongst the village tribes of savages. To do this you need food, water, and shelter. Hunt animals, plant crops, tame and breed animals, make weapons, armors and potions, raid villages, defend your base, fight with big creatures...",MightByte Games,MightByte Games,"Action, Adventure, Casual, Indie","{'Survival': 255, 'PvE': 249, 'Action': 246, 'Open World': 243, 'Co-op': 240, 'Adventure': 233, 'Hunting': 231, 'Medieval': 230, 'Base-Building': 227, 'Indie': 222, 'First-Person': 215, 'Early Access': 210, 'Singleplayer': 198, 'Realistic': 191, 'Crafting': 184, 'Casual': 175, 'Open World Survival Craft': 171, 'Cycling': 166, 'Dungeon Crawler': 166, 'Nature': 163}",game,"[Multi-player, Single-player, Co-op, Online Co-op, LAN Co-op]","0 .. 20,000",37,18,899,899,0,5,"English, Russian, Simplified Chinese","{'windows': True, 'mac': False, 'linux': False}",2021/12/25,0,https://www.instagram.com/mightbyte_games/,https://cdn.akamai.steamstatic.com/steam/apps/934180/header.jpg?t=1666776693
47400,770460,The Dummy Experiment,"Scientist Strohmann loves to play around with all kinds of crazy stuff. This time he thought of a cruel and inhumane experiment in which his test subject, Dummy 44, will experience a horrible and terrifying pain. Your role is to help Dummy 44 to escape the laboratory forever.","Kai van Ommeren, Sedat Bilen, Niek Oosterbaan, Wesley Kroon, Martijn de Vree, Max Geraci, Plaisir Nzuzi, Kevin van Huit",Cetus Studios,"Adventure, Free to Play, Indie","{'Free to Play': 36, 'Indie': 25, 'Adventure': 24, 'Difficult': 16, 'Multiplayer': 15}",game,"[Single-player, Captions available, Commentary available, Stats, Steam Leaderboards, Full controller support]","20,000 .. 50,000",158,88,0,0,0,0,English,"{'windows': True, 'mac': True, 'linux': False}",2018/01/19,0,https://cetusstudios.com/the-dummy-experiment/,https://cdn.akamai.steamstatic.com/steam/apps/770460/header.jpg?t=1530132030
16783,1539360,Charlie The Roach,A hilarious take on an old roach named Charlie who needs to platform his way through various environments to free his friends from an evil one.,Gray Mantis,Gray Mantis,Action,"{'Action': 72, 'Action-Adventure': 61, 'Platformer': 58, '2D Platformer': 55, 'Side Scroller': 52, 'Cartoony': 46, 'Stylized': 44, 'Comedy': 42, 'Dark Humor': 39, 'Family Friendly': 37, 'Funny': 33, 'Old School': 31, 'Controller': 29, '2D': 27, 'Singleplayer': 26}",game,"[Single-player, Steam Achievements]","0 .. 20,000",3,0,299,299,0,0,English,"{'windows': True, 'mac': False, 'linux': False}",2021/03/5,0,http://www.graymantis.com/games/charlietheroach,https://cdn.akamai.steamstatic.com/steam/apps/1539360/header.jpg?t=1667020208
25101,1937980,DUST - A Post Apocalyptic RPG,"A post apocalyptic survival game with randomly generated events, that will challenge your decision making. It is inspired by classic games such as: Fallout and Wasteland series. Every decision you make in the game has consequences that can result in success or death, both in short term or long term.",SEGEDI INC.,SEGEDI INC.,"Adventure, Indie, RPG, Simulation","{'Exploration': 292, 'Tactical RPG': 286, 'Turn-Based Tactics': 282, 'Interactive Fiction': 276, 'Souls-like': 275, 'Visual Novel': 268, 'PvE': 266, 'Action RPG': 264, 'RPG': 262, 'Rogue-like': 258, 'Cold War': 256, 'World War I': 251, '2D': 240, 'Military': 238, 'World War II': 234, 'Stealth': 231, 'Artificial Intelligence': 226, 'Robots': 216, 'Survival': 214, 'War': 200}",game,[Single-player],"20,000 .. 50,000",1,2,599,599,0,0,English,"{'windows': True, 'mac': False, 'linux': False}",2022/05/1,0,https://segediinc.com/dust-a-post-apocalyptic-role-playing-game,https://cdn.akamai.steamstatic.com/steam/apps/1937980/header.jpg?t=1651392722
54

In [4]:
# wilson score for finding ratings from positive and negative votes
from math import sqrt
def wilson(p, n, z = 1.96):
    denominator = 1 + z**2/n
    centre_adjusted_probability = p + z*z / (2*n)
    adjusted_standard_deviation = sqrt((p*(1 - p) + z*z / (4*n)) / n)
    
    lower_bound = (centre_adjusted_probability - z*adjusted_standard_deviation) / denominator
    upper_bound = (centre_adjusted_probability + z*adjusted_standard_deviation) / denominator
#     return (lower_bound, upper_bound)
    return ((lower_bound + upper_bound)/2)

In [10]:
# taking a quick look at everything
display(steam_2022.sample(2))

,appid,name,short_description,developer,publisher,genre,tags,type,categories,owners,positive,negative,total_votes,rating,price,initialprice,discount,ccu,languages,platforms,release_date,required_age,website,header_image
4659,1133780,Bandit the game,"Bandit the game is an RPG-survival with an open world and elements of tower defense, where you can build own base, gang and vehicle.",Adrian Ubiorski,Adrian Ubiorski,"Action, Indie, RPG, Early Access","{'Early Access': 156, 'Survival': 141, 'Exploration': 135, 'Base-Building': 131, 'Tower Defense': 129, 'Indie': 125, 'Sandbox': 115, 'Fantasy': 113, 'Open World': 108, 'RPG': 105, 'Action': 103, '3D': 101, 'Third Person': 97, 'Building': 90, 'Walking Simulator': 84, 'Crafting': 79, 'Combat': 77, 'Inventory Management': 73, 'Resource Management': 70, 'Singleplayer': 65}",game,"[Single-player, Partial Controller Support, Steam Cloud]","20,000 .. 50,000",15,1,0,0,0,0,0,0,"English, Polish","{'windows': True, 'mac': False, 'linux': False}",2019/08/28,0,https://adrianubiorski.wixsite.com/bandit,https://cdn.akamai.steamstatic.com/steam/apps/1133780/header.jpg?t=1643737254
12251,1380260,Space Empires: Starfury,In Space Empires: Starfury you are on a wild intergalactic ride through the Space Empires universe. You are an undercover agent for the TCN (Terran Confederation Navy) trying to determine the cause of a recent escalation of pirate attacks in the Neutral territories.,Malfador Machinations,Strategy First,"Action, Strategy","{'Strategy': 71, 'Space': 49, 'Real Time Tactics': 42, 'RTS': 36, 'Space Sim': 32, 'Trading': 32, 'Management': 29, '3D': 26, 'Action': 24, 'Sci-fi': 23, 'Simulation': 22, 'Aliens': 19, 'Futuristic': 18, 'Combat': 16, 'Singleplayer': 14}",game,[Single-player],"20,000 .. 50,000",7,2,0,0,999,999,0,1,English,"{'windows': True, 'mac': False, 'linux': False}",2020/08/3,0,,https://cdn.akamai.steamstatic.com/steam/apps/1380260/header.jpg?t=1596469356


In [27]:
# Converting tags dictionary to list only, and getting rid of those numbers - whatever thhey mean.
# changing total_votes column as well
# and the average rating
for index, row in steam_2022.iterrows():
    current_row_tags = row['tags']
#     current_row_tags = list(current_row_tags.keys())
#     steam_2022.loc[index,'tags'] = current_row_tags

for index,row in steam_2022.iterrows():
    current_row_total_votes = row['positive'] + row['negative']
    steam_2022.loc[index, 'total_votes'] = current_row_total_votes
    positive_votes = row['positive']
    if (current_row_total_votes != 0) & (positive_votes != 0):
        current_row_rating = wilson((positive_votes/current_row_total_votes), current_row_total_votes)
        steam_2022.loc[index, 'rating'] = round(current_row_rating*100,2)
    
display(steam_2022.sample(5))

,appid,name,short_description,developer,publisher,genre,tags,type,categories,owners,positive,negative,total_votes,rating,price,initialprice,discount,ccu,languages,platforms,release_date,required_age,website,header_image
44375,690580,Evil Spirits,"Tom and his friends went on vacation on a beautiful tropical island. Where they met a tribe of friendly Aborigines, the leader of the natives told a wonderful story that will change their whole life.",Exciting Games,Exciting Games,"Action, Adventure, Simulation","{'Action': 31, 'Adventure': 31, 'Simulation': 20, 'Horror': 13, 'Violent': 13, 'FPS': 12, 'Singleplayer': 11, 'Funny': 10, 'Shooter': 10, 'Survival': 10, 'Zombies': 10, 'Atmospheric': 9, 'Great Soundtrack': 9, 'Kickstarter': 9, 'Linear': 9, 'Military': 9, 'Moddable': 9, 'Modern': 9, 'Puzzle': 9, 'War': 9}",game,"[Single-player, Steam Achievements]","0 .. 20,000",18,27,45,40.79,99,99,0,0,"English, French, Italian, German, Spanish - Spain, Arabic, Bulgarian, Portuguese - Brazil, Hungarian, Vietnamese, Greek, Danish, Spanish - Latin America, Traditional Chinese, Simplified Chinese, Korean, Dutch, Norwegian, Polish, Portuguese - Portugal, Romanian, Russian, Thai, Turkish, Ukrainian, Finnish, Czech, Swedish, Japanese","{'windows': True, 'mac': False, 'linux': False}",2017/08/24,0,,https://cdn.akamai.steamstatic.com/steam/apps/690580/header.jpg?t=1550659061
50073,841190,"Honey, I Joined a Cult","It's time to start working towards that ultimate goal of enlightenment, faith and money…lots and lots of money! Become the ultimate tycoon - create, customise, and manage your own cult whilst listening to funky music. Welcome to Honey, I Joined a Cult!",Sole Survivor Games,Team17,"Casual, Indie, Simulation, Strategy","{'Simulation': 362, 'Base-Building': 357, 'Resource Management': 356, 'Management': 349, 'Illuminati': 346, 'Strategy': 310, 'Sandbox': 304, 'Funny': 279, 'Faith': 276, 'Comedy': 270, 'Building': 266, 'Political Sim': 252, 'Colony Sim': 233, 'Singleplayer': 232, 'Pixel Graphics': 226, 'Psychological': 224, 'God Game': 205, 'Dark Humor': 194, '2D': 184, 'Dark Comedy': 178}",game,"[Single-player, Steam Cloud]","50,000 .. 100,000",752,178,930,80.73,1473,2199,33,301,"English, German, Spanish - Spain, Russian, Simplified Chinese, French, Italian, Korean, Polish, Portuguese - Portugal, Traditional Chinese, Turkish, Japanese","{'windows': True, 'mac': False, 'linux': False}",2022/11/3,0,https://www.team17.com/introducing-honey-i-joined-a-cult/honey-i-joined-a-cult,https://cdn.akamai.steamstatic.com/steam/apps/841190/header.jpg?t=1668103132
22590,18020,"Precipice of Darkness, Episode Two","On the Rain-Slick Precipice of Darkness, Episode Two is the second installment in the RPG-adventure game series based on the web comic Penny Arcade. Whether you're a fan of the first installment or new to the series, you'll be able to pick up and play Episode Two as a standalone experience.",Hothead Games,Hothead Games,"Adventure, RPG","{'RPG': 46, 'Adventure': 41, 'Comedy': 20, 'Singleplayer': 11, 'Funny': 8, 'Turn-Based': 7}",game,"[Single-player, Steam Achievements]","100,000 .. 200,000",145,29,174,82.61,799,799,0,0,English,"{'windows': True, 'mac': False, 'linux': False}",2008/11/7,0,http://www.hotheadgames.com/pa.php,https://cdn.akamai.steamstatic.com/steam/apps/18020/header.jpg?t=1447351546
41211,602770,Clatter,"Create and command your own customizable squads of fighting robots in this turn-based, arena tactics game.",Facepunch Studios,Facepunch Studios,"Indie, Strategy","{'Strategy': 75, 'Indie': 70, 'Turn-Based': 29, 'Turn-Based Combat': 28, 'Turn-Based Strategy': 27, 'Turn-Based Tactics': 27, 'Tactical': 25, 'Multiplayer': 14, 'Cute': 13, 'Character Customization': 12}",game,"[Multi-player, Single-player, Steam Achievements, Steam Cloud, Steam Trading Cards, In-App Purchases, Online PvP, PvP]","50,000 .. 100,000",498,90,588,84.47,0,0,0,5,English,"{'windows': True, 'mac': False, 'linux': False}",2018/12/10,0,https://clatter.facepunch.com/,ht